#TÓM TẮT CÁC BƯỚC CHÍNH CỦA THUẬT TOÁN ACTOR-CRITIC VỚI GAE
# 1. **Khởi tạo môi trường và siêu tham số**:
#    - Tạo môi trường "CartPole-v1" và định nghĩa các tham số cần thiết như gamma, lambda, số bước tối đa, số tập tối đa, v.v.
# 2. **Xây dựng mô hình Actor-Critic**:
#    - Mô hình gồm 2 phần:
#      - **Actor**: Dự đoán xác suất chọn các hành động.
#      - **Critic**: Dự đoán giá trị của trạng thái hiện tại.
# 3. **Huấn luyện mô hình**:
#    - Trong mỗi tập (episode), mô hình:    
*   Chọn hành động dựa trên chính sách (policy).
*   Thực hiện hành động trong môi trường và lưu thông tin phần thưởng, giá trị trạng thái.    
#    - Tính toán Advantage và Returns bằng GAE (Generalized Advantage Estimation).
#    - Tính toán mất mát Actor, Critic, và Entropy để cập nhật trọng số mô hình.
# 4. **Điều chỉnh độ nhiễu Entropy**:
#    - Giảm dần Entropy Beta dựa trên phần thưởng trung bình để chuyển từ giai đoạn khám phá sang khai thác.
# 5. **Theo dõi tiến trình và kết thúc huấn luyện**:
#    - Dừng huấn luyện khi phần thưởng trung bình đạt ngưỡng hoặc đạt số tập tối đa.

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # Thiết lập backend sử dụng TensorFlow
import gym  # Thư viện mô phỏng môi trường
import numpy as np  # Xử lý số liệu
import keras
from keras import ops  # Công cụ toán học cho TensorFlow
from keras import layers  # Định nghĩa các lớp mạng nơ-ron
import tensorflow as tf  # Framework học sâu

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [ ]:
# ==================== BƯỚC 1: KHỞI TẠO MÔI TRƯỜNG VÀ CẤU HÌNH ====================
# Tạo môi trường CartPole-v1
env = gym.make("CartPole-v1")
# Định nghĩa các siêu tham số
num_inputs = 4  # Số lượng đầu vào (trạng thái)
num_actions = 2  # Số hành động có thể thực hiện
num_hidden1 = 256  # Số lượng nơ-ron trong lớp ẩn đầu tiên
num_hidden2 = 128  # Số lượng nơ-ron trong lớp ẩn thứ hai
gamma = 0.97  # Hệ số chiết khấu
lambda_ = 0.95  # Hệ số GAE
max_steps_per_episode = 500  # Số bước tối đa mỗi tập
max_episodes = 1000  # Số tập tối đa
initial_entropy_beta = 0.01  # Hệ số entropy ban đầu
learning_rate = 0.001  # Tốc độ học

In [ ]:
# ==================== BƯỚC 2: XÂY DỰNG MÔ HÌNH ACTOR-CRITIC ====================
# Định nghĩa các lớp và đầu ra cho Actor và Critic
inputs = layers.Input(shape=(num_inputs,))  # Đầu vào là trạng thái
common = layers.Dense(num_hidden1, activation="relu")(inputs)  # Lớp ẩn 1 với hàm kích hoạt ReLU
common = layers.LayerNormalization()(common)  # Chuẩn hóa đầu ra lớp ẩn 1
common = layers.Dense(num_hidden2, activation="relu")(common)  # Lớp ẩn 2 với hàm kích hoạt ReLU
action = layers.Dense(num_actions, activation="softmax")(common)  # Lớp đầu ra cho Actor (xác suất hành động)
critic = layers.Dense(1)(common)  # Lớp đầu ra cho Critic (giá trị trạng thái)
# Kết hợp các đầu ra để tạo mô hình Actor-Critic
model = tf.keras.Model(inputs=inputs, outputs=[action, critic])

# Định nghĩa hàm tối ưu hóa và hàm mất mát
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Sử dụng Adam Optimizer
huber_loss = tf.keras.losses.Huber()  # Hàm mất mát Huber


In [ ]:
# Hàm tính toán GAE (Generalized Advantage Estimation) và Returns
def compute_gae_and_returns(rewards, values, gamma=0.97, lambda_=0.95):
    advantages = []  # Lưu trữ Advantage
    gae = 0  # Giá trị GAE ban đầu
    returns = []  # Lưu trữ Returns
    # Duyệt ngược qua danh sách phần thưởng để tính toán GAE
    for t in reversed(range(len(rewards))):
        # Tính delta (phần sai lệch)
        delta = rewards[t] + gamma * (values[t + 1] if t + 1 < len(values) else 0) - values[t]
        gae = delta + gamma * lambda_ * gae  # Tính giá trị GAE
        advantages.insert(0, gae)  # Thêm Advantage vào danh sách
        returns.insert(0, gae + values[t])  # Tính Return
    return np.array(advantages), np.array(returns)  # Trả về Advantage và Returns

In [ ]:
# ==================== BƯỚC 3: HUẤN LUYỆN MÔ HÌNH ====================
running_reward = 0  # Theo dõi phần thưởng trung bình
entropy_beta = initial_entropy_beta  # Hệ số entropy ban đầu

# Vòng lặp huấn luyện qua các tập (episodes)
for episode_count in range(1, max_episodes + 1):
    state = env.reset()  # Khởi tạo trạng thái
    episode_reward = 0  # Tổng phần thưởng mỗi tập
    action_probs_history = []  # Lưu lịch sử xác suất hành động
    critic_value_history = []  # Lưu lịch sử giá trị Critic
    rewards_history = []  # Lưu lịch sử phần thưởng

    with tf.GradientTape() as tape:  # Bắt đầu ghi lại gradient
        for step in range(max_steps_per_episode):
            # Chuyển trạng thái thành tensor để đưa vào mô hình
            state = tf.convert_to_tensor(state, dtype=tf.float32)
            state = tf.expand_dims(state, axis=0)

            # Lựa chọn hành động từ Actor và tính giá trị Critic
            action_probs, critic_value = model(state)
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))  # Chọn hành động ngẫu nhiên dựa trên xác suất

            action_probs_history.append(tf.math.log(action_probs[0, action]))  # Lưu log xác suất hành động
            critic_value_history.append(critic_value[0, 0])  # Lưu giá trị Critic

            # Thực hiện hành động và nhận phản hồi từ môi trường
            state, reward, done, _ = env.step(action)
            rewards_history.append(reward)  # Lưu phần thưởng
            episode_reward += reward  # Cộng dồn phần thưởng

            if done:  # Kết thúc tập nếu trạng thái kết thúc
                break

        # Thêm giá trị cuối cùng để tính GAE
        critic_value_history.append(0 if done else model(tf.expand_dims(state, axis=0))[1][0, 0])

        # Tính Advantage và Returns
        advantages, returns = compute_gae_and_returns(rewards_history, critic_value_history, gamma, lambda_)

        # Chuẩn hóa Advantage để giảm phương sai
        advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-8)

        # Tính toán mất mát Actor, Critic và Entropy
        actor_losses = [-log_prob * adv for log_prob, adv in zip(action_probs_history, advantages)]
        critic_losses = [huber_loss(tf.expand_dims(v, 0), tf.expand_dims(r, 0)) for v, r in zip(critic_value_history[:-1], returns)]
        entropy_loss = -entropy_beta * tf.reduce_mean(tf.reduce_sum(action_probs * tf.math.log(action_probs + 1e-10), axis=1))

        # Tổng hợp các loại mất mát
        loss_value = tf.reduce_sum(actor_losses) + 0.5 * tf.reduce_sum(critic_losses) + entropy_loss

        # Tính gradient và cập nhật trọng số mô hình
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Theo dõi phần thưởng trung bình
    running_reward = 0.1 * episode_reward + 0.9 * running_reward

    # Điều chỉnh Entropy Beta để giảm độ nhiễu
    entropy_beta = max(0.001, initial_entropy_beta * (1 - running_reward / 500))

    # In kết quả mỗi 10 tập
    if episode_count % 10 == 0:
        print(f"Episode: {episode_count}, Running Reward: {running_reward:.2f}")

    # Dừng huấn luyện nếu đạt ngưỡng phần thưởng trung bình
    if running_reward > 475:
        print(f"Đạt kết quả sau {episode_count} tập!")
        break

# Kết thúc huấn luyện nếu đạt số tập tối đa
if episode_count == max_episodes:
    print(f"Đã huấn luyện xong sau {max_episodes} tập.")

Episode: 10, Running Reward: 6.63
Episode: 20, Running Reward: 8.84
Episode: 30, Running Reward: 10.44
Episode: 40, Running Reward: 30.46
Episode: 50, Running Reward: 56.09
Episode: 60, Running Reward: 95.94
Episode: 70, Running Reward: 122.78
Episode: 80, Running Reward: 132.42
Episode: 90, Running Reward: 234.89
Episode: 100, Running Reward: 192.00
Episode: 110, Running Reward: 254.86
Episode: 120, Running Reward: 145.21
Episode: 130, Running Reward: 136.94
Episode: 140, Running Reward: 152.56
Episode: 150, Running Reward: 258.28
Episode: 160, Running Reward: 237.88
Episode: 170, Running Reward: 227.89
Episode: 180, Running Reward: 301.63
Episode: 190, Running Reward: 430.83
Episode: 200, Running Reward: 363.27
Episode: 210, Running Reward: 376.79
Episode: 220, Running Reward: 400.64
Episode: 230, Running Reward: 412.40
Episode: 240, Running Reward: 428.00
Episode: 250, Running Reward: 445.14
Episode: 260, Running Reward: 402.52
Episode: 270, Running Reward: 274.41
Episode: 280, Runn